Connect to Google Drive:

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/MP'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/MP


Import required modules:

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
import torch

Set seed for PyTorch random number generation:

In [3]:
torch.random.manual_seed(0)

Open the original dataset and print its column:

In [4]:
#df = pd.read_csv("Datasets/new_dataset.csv", encoding="utf-16", sep="\t")
df = pd.read_csv("Datasets/claim_review.csv")
print(df.columns)

Index(['id', '@context', '@type', 'claimReviewed', 'datePublished', 'url',
       'author.@type', 'author.name', 'author.url', 'itemReviewed.@type',
       'itemReviewed.author.name', 'itemReviewed.author.@type',
       'itemReviewed.author.sameAs', 'reviewRating.@type',
       'reviewRating.ratingValue', 'reviewRating.alternateName',
       'author.image', 'itemReviewed.name', 'itemReviewed.datePublished',
       'itemReviewed.firstAppearance.url', 'itemReviewed.firstAppearance.type',
       'itemReviewed.author.image', 'reviewRating.ratingExplanation',
       'itemReviewed.author.jobTitle', 'reviewRating.bestRating',
       'reviewRating.worstRating', 'reviewRating.image',
       'itemReviewed.appearance.1.url', 'itemReviewed.appearance.1.@type',
       'itemReviewed.appearance.2.url', 'itemReviewed.appearance.2.@type',
       'itemReviewed.appearance.3.url', 'itemReviewed.appearance.3.@type',
       'itemReviewed.appearance.4.url', 'itemReviewed.appearance.4.@type',
       'itemRevi

<ipython-input-4-8506a802bca8>:2: DtypeWarning: Columns (14,24,25,45,46,47,48,49,50,51,52,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Datasets/claim_review.csv")


Load the model:

In [5]:
#model_name = "microsoft/Phi-3-mini-4k-instruct"
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    trust_remote_code=True,
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name, device = "cuda")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setup input arguments for the LLM>

In [6]:
system_prompt = """You are an assistant that helps fact-checking claims.
  Given a claim and the label telling its veracity, you need to normalize the label to a standart format,
  between this choices: True, Mostly true, Mixture, Mostly False or False. Answer only with the label.
  For example:
  Claim: The Eiffel Tower is located in Germany
  Label: Totally False
  """

pipe = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
)

generation_args = {
  "max_new_tokens": 4,
  "return_full_text": False,
  "do_sample": False,
}

Map labels:

In [7]:
for i in range(25, 49):

  claim = df["claimReviewed"].iloc[i]
  label = df["reviewRating.alternateName"].iloc[i]

  claim_review = f"Claim: {claim}\n Label: {label}"

  messages = [
      {"role": "user", "content": system_prompt},
      {"role": "assistant", "content": "False"},
      {"role": "user", "content": "Claim: One plus one equals two\n Label: True"},
      {"role": "assistant", "content": "True"},
      {"role": "user", "content": claim_review},
  ]

  output = pipe(messages, **generation_args)

  print(f"Messages is {messages}")
  print(f"Claim_review is {claim_review}")
  print("Output of the model is: " + output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Messages is [{'role': 'user', 'content': 'You are an assistant that helps fact-checking claims.\n  Given a claim and the label telling its veracity, you need to normalize the label to a standart format,\n  between this choices: True, Mostly true, Mixture, Mostly False or False. Answer only with the label.\n  For example:\n  Claim: The Eiffel Tower is located in Germany\n  Label: Totally False\n  '}, {'role': 'assistant', 'content': 'False'}, {'role': 'user', 'content': 'Claim: One plus one equals two\n Label: True'}, {'role': 'assistant', 'content': 'True'}, {'role': 'user', 'content': 'Claim: යක්ෂයන්ගේ මුහුණු සහිත කමිස හැදි සජබ මන්ත්\u200dරීවරු ?\n Label: Altered'}]
Claim_review is Claim: යක්ෂයන්ගේ මුහුණු සහිත කමිස හැදි සජබ මන්ත්‍රීවරු ?
 Label: Altered
Output of the model is: Mixture


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Messages is [{'role': 'user', 'content': 'You are an assistant that helps fact-checking claims.\n  Given a claim and the label telling its veracity, you need to normalize the label to a standart format,\n  between this choices: True, Mostly true, Mixture, Mostly False or False. Answer only with the label.\n  For example:\n  Claim: The Eiffel Tower is located in Germany\n  Label: Totally False\n  '}, {'role': 'assistant', 'content': 'False'}, {'role': 'user', 'content': 'Claim: One plus one equals two\n Label: True'}, {'role': 'assistant', 'content': 'True'}, {'role': 'user', 'content': "Claim: De presidentsverkiezingen in Frankrijk zijn gefraudeerd door stembiljetten voor Le Pen lichtjes te verscheuren.\n Label: In het noorden van Frankrijk werden inderdaad licht beschadigde stembiljetten gesignaleerd. Dat is echter geen bewijs van opzettelijke verkiezingsfraude. Ten eerste waren er in de stembureaus andere biljetten beschikbaar. Ten tweede is zo'n lichte beschadeging zoals te zien in 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Messages is [{'role': 'user', 'content': 'You are an assistant that helps fact-checking claims.\n  Given a claim and the label telling its veracity, you need to normalize the label to a standart format,\n  between this choices: True, Mostly true, Mixture, Mostly False or False. Answer only with the label.\n  For example:\n  Claim: The Eiffel Tower is located in Germany\n  Label: Totally False\n  '}, {'role': 'assistant', 'content': 'False'}, {'role': 'user', 'content': 'Claim: One plus one equals two\n Label: True'}, {'role': 'assistant', 'content': 'True'}, {'role': 'user', 'content': 'Claim: Bispo do Porto: abuso [de menores] "não é um crime público". É falso\n Label: Falso'}]
Claim_review is Claim: Bispo do Porto: abuso [de menores] "não é um crime público". É falso
 Label: Falso
Output of the model is: False


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Messages is [{'role': 'user', 'content': 'You are an assistant that helps fact-checking claims.\n  Given a claim and the label telling its veracity, you need to normalize the label to a standart format,\n  between this choices: True, Mostly true, Mixture, Mostly False or False. Answer only with the label.\n  For example:\n  Claim: The Eiffel Tower is located in Germany\n  Label: Totally False\n  '}, {'role': 'assistant', 'content': 'False'}, {'role': 'user', 'content': 'Claim: One plus one equals two\n Label: True'}, {'role': 'assistant', 'content': 'True'}, {'role': 'user', 'content': 'Claim: There are no penal provisions in place against illegal migrants in India\n Label: FALSE'}]
Claim_review is Claim: There are no penal provisions in place against illegal migrants in India
 Label: FALSE
Output of the model is: False


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Messages is [{'role': 'user', 'content': 'You are an assistant that helps fact-checking claims.\n  Given a claim and the label telling its veracity, you need to normalize the label to a standart format,\n  between this choices: True, Mostly true, Mixture, Mostly False or False. Answer only with the label.\n  For example:\n  Claim: The Eiffel Tower is located in Germany\n  Label: Totally False\n  '}, {'role': 'assistant', 'content': 'False'}, {'role': 'user', 'content': 'Claim: One plus one equals two\n Label: True'}, {'role': 'assistant', 'content': 'True'}, {'role': 'user', 'content': 'Claim: ఆల్కహాల్ మరియు ధూమపానం కలిపితే వచ్చే నష్టాల కంటే పామ్ ఆయిల్\u200c వల్ల వచ్చే నష్టాలే ఎక్కువ\n Label: MISLEADING'}]
Claim_review is Claim: ఆల్కహాల్ మరియు ధూమపానం కలిపితే వచ్చే నష్టాల కంటే పామ్ ఆయిల్‌ వల్ల వచ్చే నష్టాలే ఎక్కువ
 Label: MISLEADING
Output of the model is: Mostly False
Messages is [{'role': 'user', 'content': 'You are an assistant that helps fact-checking claims.\n  Given a claim and t